# Shear Dispersion IVP Solution for a single Mode:


The differential equation is

\begin{equation}
\epsilon\frac{\partial\theta}{\partial t} + \cos{(y)}\frac{\partial\theta}{\partial x} = \frac{1}{Pe}\frac{\partial^2\theta}{\partial x^2} + \epsilon\frac{\partial^2\theta}{\partial y^2}
\end{equation}


with the initial condition

\begin{equation}
\theta(x, y, 0) = \cos{(k_{0}x)} .
\end{equation}

The solution is:
\begin{equation}
\theta(x, y, t) = \theta_{0}Re\left\{\sum_{n=0}^{\infty}2A_{0}^{(2n)}ce_{2n}\exp{\left[ ik_{0}x - \left(\frac{a_{2n}}{4} + \frac{k_{0}^2}{\epsilon Pe} \right)t\right]}\right\}
\end{equation}





In [ ]:
import numpy as np
import xarray as xr
from mathieu_functions import A_coefficients
from mathieu_functions import mathieu_functions as mfs
import holoviews as hv
hv.extension('bokeh')

In [ ]:
# =================================
# Geometry definitions
# =================================
L = np.pi # Channel width (tilde{y}-direction)
alpha = 10  # Sets the length of the channel, and thus smallest mode that fits the x-periodic domain (k_min = 1/alpha)
Nx = 500  # length of x-array 

x = np.linspace(-alpha * L, alpha * L, Nx)
y = np.linspace(0, L, Nx//5)
X, Y = np.meshgrid(x, y)

# =================================
# Time, Non-dimensional, scales like diffusive timescale (not seconds but rather units of diffusive timescales).
# The time array below, works for k=1/10. A larger k may decay faster, thus need to modify t-array.
# =================================
t = np.linspace(0, 2.5, 200)

# =============================
#  Physical Parameters. 
#  You can change this. But must run cells again to calculate the eigenvals and eigenfunctions.
# =============================
eps = 0.05  # The smaller, the more EPs the solution will incorporate
k = 0.1 # lowest possible is 1 / alpha. 
Pe = 1 / eps  # This implies \epsilon\Pe = 1. But other definitions can be used too.


# =============================
#  Mathieu Canonical Parameter (define as an array but will only use the last value)
# =============================
qf = 2 * k / eps  ## The larger its magnitude (when compared to q = 1i), the faster solutions will decay, and the larger they will disperse.
Q = np.linspace(0, qf, 100)* (1j)

## Check value of Mathieu Parameter
The larger the value (e.g. $|q|>100$) the larger the Matrix size (N value below). 

Good rule of thumbs:
* if $q>100i$, then $N=50$. 
* If $q>500i$ then $N=75$. 

The larger N, the slower the code and mode memory use.

In [ ]:
if qf > 1000:
    print('Value of parameter q is:', (qf * (1j)))
    raise Warning('Change either epsilon or k, to reduce the size of q. The current code only works for values q>1000i')
print('Value of parameter q is:', (qf * (1j)))

In [ ]:
print('Mode of initial condition (k_0):', k)

## Evaluate Mathieu Eigenvalue System 


In [ ]:
N = 25  # matrix size 
As = A_coefficients(Q, N, 'even', 'one')
Eig_fns = mfs.ce_even(Q, y, N, As=As)

## Initialize a list of Mathieu Eigenfns with correct size

In [ ]:
CE = []  # Initialize list containing Mathieu Eigenfunctions
for n in range(N // 2):
    ce = np.repeat(Eig_fns['ce'+str(2 * n)][:, :, np.newaxis], Nx, axis=2)
    CE.append(ce)

## Construct the solution to the IVP

In [ ]:
## Initialize the array
CE2n = 0
for n in range(N//2):
    CE2n = CE2n + 2 * As['A'+str(2 * n)][-1, 0] * CE[n][-1, :, :]
T0 = np.real(np.exp((k * X) * (1j)) * CE2n)
T0 = T0[np.newaxis, :]


## Add temporal contribution
for i in range(1, len(t)):
    CE2n = 0
    for n in range(N//2):
        CE2n = CE2n + 2 * As['A'+str(2 * n)][-1, 0] * CE[n][-1, :, :] * np.exp(-(0.25 * As['a'+str(2 * n)][-1] + k**2) * t[i])
    t0 = np.real(np.exp((k * X) * (1j)) * CE2n)
    t0 = t0[np.newaxis, :]
    T0 = np.append(T0, t0, axis=0)

## Make Animation

In [ ]:
coords = {"time": t, 
          "y": y, 
          "x": x}
Temp = xr.DataArray(T0, coords=coords, dims=["time", 'y', 'x'])
ds = xr.Dataset({'Theta': Temp})

In [ ]:
%%output holomap='scrubber'
%%opts Image style(cmap='PRGn') plot[colorbar=True]
%%opts Image [width=600, height=450]
hv_ds = hv.Dataset(ds.Theta.isel(time=slice(0, -1, 4), x=slice(0,-1, 2)))
hv_ds.to(hv.Image, ['x', 'y'])